In [1]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
#from torchvision.datasets import MNIST
#import torchvision.transforms as transforms
from torch.utils.data import Dataset
import spacy
#import pytorch_lightning as pl
#from pytorch_lightning import Trainer
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg
import scipy
from sklearn.manifold import TSNE
import sklearn
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.svm import LinearSVC, SVR
from sklearn.multioutput import MultiOutputRegressor
#from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
import scipy.io
import random
import inlp
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from typing import Tuple, Dict
from typing import List
import tqdm
import mnist_reader
import urllib
import time
%load_ext autoreload
%autoreload

In [34]:
def load_wiki(url = "http://nlp.biu.ac.il/~ravfogs/resources/wiki.1million.raw.txt"):
    
  response = urllib.request.urlopen(url)
  raw = response.read().decode('utf8')
  sents = raw.split("\n")
  
  return sents[:]


def create_docs(sentences: List[str]) -> List[spacy.tokens.Doc]:

    #parsing and toekenizing (NOTE: takes about 10 minutes)
    
    nlp = spacy.load('en_core_web_sm')
    nlp.remove_pipe("ner")
    
    start = time.time()
    
    docs = list(nlp.pipe(sentences, batch_size = 100))
    
    docs = list(docs)
    print(time.time() - start)
    
    return docs


def create_dataset_for_related_words(docs, num_samples = 100*1e3):
    
    data = []
    pbar = tqdm.tqdm(total = num_samples)
    
    while len(data) < num_samples:
        
        doc = random.choice(docs)
        sent = " ".join([tok.text for tok in doc])
        if len(doc) < 9: continue
        
        for i,tok in enumerate(doc):
            pbar.update(2)
            head = tok.head
            children = list(tok.children)
            related = [head] + children
            random_related = random.choice(related)
            data.append((sent, i, random_related.i, 1))
            
            # choose negative example
            
            chosen = False
            while not chosen:
                
                random_word = random.choice(doc)
                if random_word not in related:
                    chosen = True
                    data.append((sent, i, random_word.i, 0))
    
    random.shuffle(data)
    return data
                
    
def create_dataset_for_depth(docs, num_samples = 25*1e3):
    
    data = []
    pbar = tqdm.tqdm(total = num_samples)
    i = 0
    
    while len(data) < num_samples:
        pbar.update(1)
        i += 1
        doc = random.choice(docs)
        sent = " ".join([tok.text for tok in doc])
        if len(doc) < 9: continue
        
        for i,tok in enumerate(doc):
            depth = 0
            head = tok
            while head.dep_ != "ROOT":
                
                depth += 1
                head = head.head
                
            data.append((sent, i, depth))
    
    random.shuffle(data)
    return data        

## Collect data

In [3]:
sents = load_wiki()

In [6]:
sents = sents[:25000]
docs = create_docs(sents)

36.69035053253174


In [35]:
data = create_dataset_for_depth(docs, num_samples = 25 * 1e3)










  5%|▍         | 1212/25000.0 [00:00<00:02, 11485.79it/s]A


## Encoder

In [95]:
class BertEncoder(object):
    
    def __init__(self, device = 'cpu'):

        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')
            
        self.model.eval()
        self.model.to(device)
        self.device = device
        
    def tokenize(self, original_sentence: List[str]) -> Tuple[List[str], Dict[int, int]]:

        """
        Parameters
        ----------
        Returns
        -------
        bert_tokens: The sentence, tokenized by BERT tokenizer.
        orig_to_tok_map: An output dictionary consisting of a mapping (alignment) between indices in the original tokenized sentence, and indices in the sentence tokenized by the BERT tokenizer. See https://github.com/google-research/bert
        """

        bert_tokens = ["[CLS]"]
        orig_to_tok_map = {}
        has_subwords = False
        is_subword = []

        for i, w in enumerate(original_sentence):
            tokenized_w = self.tokenizer.tokenize(w)
            has_subwords = len(tokenized_w) > 1
            is_subword.append(has_subwords)
            bert_tokens.extend(tokenized_w)

            orig_to_tok_map[i] = len(bert_tokens) - 1

        bert_tokens.append("[SEP]")
        
        return (bert_tokens, orig_to_tok_map)

    
    def encode(self, sentence: str):
        
        tokenized_text, orig2tok = self.tokenize(sentence.split(" "))
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens]).to(self.device)

        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0].detach().cpu().numpy()
            return (predictions.squeeze(), orig2tok)
        

def collect_bert_states_for_grammatical_relation(bert, data: List[Tuple]):
    
    
    for i,d in tqdm.tqdm(enumerate(data), total = len(data)):
        
        sent = d[0]
        
        j,k = d[1], d[2]
        label = d[3]
        
        preds, orig2tok = bert.encode(sent)
        x1,x2 = preds[orig2tok[j]], preds[orig2tok[k]]
        data[i] = (sent, j, k, x1, x2, label)
    
    return data


def collect_bert_states_for_depth(bert, data: List[Tuple]):
    
    
    for i,d in tqdm.tqdm(enumerate(data), total = len(data)):
        
        sent = d[0]
        jj = d[1]
        label = d[2]
        
        preds, orig2tok = bert.encode(sent)
        v = preds[orig2tok[jj]]
        data[i] = (sent, jj, v, label)
    
    return data
        

In [96]:
bert = BertEncoder(device = "cpu")

In [137]:
data_with_states = collect_bert_states_for_depth(bert, data[:6000])










  0%|          | 0/6000 [00:00<?, ?it/s]








  0%|          | 2/6000 [00:00<06:27, 15.48it/s]








  0%|          | 5/6000 [00:00<05:47, 17.27it/s]








  0%|          | 7/6000 [00:00<06:03, 16.50it/s]








  0%|          | 9/6000 [00:00<05:55, 16.84it/s]








  0%|          | 11/6000 [00:00<06:07, 16.29it/s]








  0%|          | 13/6000 [00:00<06:03, 16.45it/s]








  0%|          | 16/6000 [00:00<05:32, 17.98it/s]








  0%|          | 18/6000 [00:00<05:30, 18.09it/s]








  0%|          | 20/6000 [00:01<05:41, 17.52it/s]








  0%|          | 22/6000 [00:01<05:44, 17.34it/s]








  0%|          | 24/6000 [00:01<05:31, 18.05it/s]








  0%|          | 26/6000 [00:01<05:31, 18.02it/s]








  0%|          | 28/6000 [00:01<05:39, 17.61it/s]








  0%|          | 30/6000 [00:01<05:47, 17.20it/s]








  1%|          | 32/6000 [00:01<05:35, 17.80it/s]








  1%|          | 35/6000 [00:01<05:12, 19.06it/s]








  1%|          | 3

  4%|▍         | 247/6000 [00:12<04:37, 20.76it/s]








  4%|▍         | 250/6000 [00:12<04:32, 21.14it/s]








  4%|▍         | 253/6000 [00:12<04:31, 21.18it/s]








  4%|▍         | 256/6000 [00:13<04:39, 20.58it/s]








  4%|▍         | 259/6000 [00:13<04:21, 21.94it/s]








  4%|▍         | 262/6000 [00:13<04:18, 22.23it/s]








  4%|▍         | 265/6000 [00:13<04:23, 21.75it/s]








  4%|▍         | 268/6000 [00:13<04:18, 22.14it/s]








  5%|▍         | 271/6000 [00:13<04:21, 21.95it/s]








  5%|▍         | 274/6000 [00:13<04:13, 22.56it/s]








  5%|▍         | 277/6000 [00:14<04:24, 21.63it/s]








  5%|▍         | 280/6000 [00:14<05:16, 18.09it/s]








  5%|▍         | 282/6000 [00:14<05:30, 17.30it/s]








  5%|▍         | 284/6000 [00:14<05:50, 16.29it/s]








  5%|▍         | 286/6000 [00:14<05:41, 16.71it/s]








  5%|▍         | 289/6000 [00:14<05:25, 17.56it/s]








  5%|▍         | 292/6000 [00:14<05:04, 18.71it/s]







  9%|▉         | 529/6000 [00:25<04:16, 21.32it/s]








  9%|▉         | 532/6000 [00:25<04:08, 21.98it/s]








  9%|▉         | 535/6000 [00:25<04:03, 22.43it/s]








  9%|▉         | 538/6000 [00:26<04:09, 21.87it/s]








  9%|▉         | 541/6000 [00:26<04:08, 21.97it/s]








  9%|▉         | 544/6000 [00:26<04:11, 21.73it/s]








  9%|▉         | 547/6000 [00:26<04:29, 20.23it/s]








  9%|▉         | 550/6000 [00:26<04:23, 20.68it/s]








  9%|▉         | 553/6000 [00:26<04:16, 21.21it/s]








  9%|▉         | 556/6000 [00:26<04:04, 22.29it/s]








  9%|▉         | 559/6000 [00:27<03:49, 23.72it/s]








  9%|▉         | 562/6000 [00:27<03:53, 23.26it/s]








  9%|▉         | 565/6000 [00:27<03:57, 22.88it/s]








  9%|▉         | 568/6000 [00:27<03:58, 22.76it/s]








 10%|▉         | 571/6000 [00:27<04:00, 22.55it/s]








 10%|▉         | 574/6000 [00:27<03:56, 22.91it/s]








 10%|▉         | 577/6000 [00:27<03:52, 23.34it/s]







 14%|█▎        | 812/6000 [00:38<04:53, 17.70it/s]








 14%|█▎        | 815/6000 [00:38<04:26, 19.48it/s]








 14%|█▎        | 818/6000 [00:38<04:16, 20.21it/s]








 14%|█▎        | 821/6000 [00:38<04:11, 20.56it/s]








 14%|█▎        | 824/6000 [00:39<03:56, 21.84it/s]








 14%|█▍        | 827/6000 [00:39<03:50, 22.49it/s]








 14%|█▍        | 830/6000 [00:39<03:46, 22.83it/s]








 14%|█▍        | 833/6000 [00:39<03:45, 22.95it/s]








 14%|█▍        | 836/6000 [00:39<03:46, 22.75it/s]








 14%|█▍        | 839/6000 [00:39<03:55, 21.95it/s]








 14%|█▍        | 842/6000 [00:39<03:42, 23.22it/s]








 14%|█▍        | 845/6000 [00:39<03:45, 22.84it/s]








 14%|█▍        | 848/6000 [00:40<03:38, 23.55it/s]








 14%|█▍        | 851/6000 [00:40<04:31, 18.94it/s]








 14%|█▍        | 854/6000 [00:40<04:19, 19.81it/s]








 14%|█▍        | 857/6000 [00:40<04:03, 21.11it/s]








 14%|█▍        | 860/6000 [00:40<03:59, 21.50it/s]







 18%|█▊        | 1092/6000 [00:51<03:55, 20.80it/s]








 18%|█▊        | 1095/6000 [00:51<03:51, 21.21it/s]








 18%|█▊        | 1098/6000 [00:51<03:51, 21.19it/s]








 18%|█▊        | 1101/6000 [00:51<03:45, 21.72it/s]








 18%|█▊        | 1104/6000 [00:52<03:53, 20.99it/s]








 18%|█▊        | 1107/6000 [00:52<03:50, 21.18it/s]








 18%|█▊        | 1110/6000 [00:52<03:52, 21.04it/s]








 19%|█▊        | 1113/6000 [00:52<03:50, 21.21it/s]








 19%|█▊        | 1116/6000 [00:52<03:50, 21.16it/s]








 19%|█▊        | 1119/6000 [00:52<03:46, 21.54it/s]








 19%|█▊        | 1122/6000 [00:52<04:01, 20.22it/s]








 19%|█▉        | 1125/6000 [00:53<03:57, 20.49it/s]








 19%|█▉        | 1128/6000 [00:53<04:05, 19.88it/s]








 19%|█▉        | 1131/6000 [00:53<04:01, 20.16it/s]








 19%|█▉        | 1134/6000 [00:53<04:06, 19.77it/s]








 19%|█▉        | 1137/6000 [00:53<03:59, 20.35it/s]








 19%|█▉        | 1140/6000 [00:53<04:03,

...

 82%|████████▏ | 4900/6000 [03:57<00:51, 21.20it/s]








 82%|████████▏ | 4903/6000 [03:57<00:49, 22.12it/s]








 82%|████████▏ | 4906/6000 [03:57<00:47, 22.97it/s]








 82%|████████▏ | 4909/6000 [03:57<00:46, 23.37it/s]








 82%|████████▏ | 4912/6000 [03:57<00:46, 23.25it/s]








 82%|████████▏ | 4915/6000 [03:57<00:47, 22.98it/s]








 82%|████████▏ | 4918/6000 [03:57<00:51, 21.04it/s]








 82%|████████▏ | 4921/6000 [03:58<00:51, 20.93it/s]








 82%|████████▏ | 4924/6000 [03:58<00:50, 21.39it/s]








 82%|████████▏ | 4927/6000 [03:58<00:49, 21.89it/s]








 82%|████████▏ | 4930/6000 [03:58<00:48, 22.09it/s]








 82%|████████▏ | 4933/6000 [03:58<00:51, 20.67it/s]








 82%|████████▏ | 4936/6000 [03:58<00:51, 20.82it/s]








 82%|████████▏ | 4939/6000 [03:58<00:49, 21.27it/s]








 82%|████████▏ | 4942/6000 [03:59<00:48, 21.66it/s]








 82%|████████▏ | 4945/6000 [03:59<00:45, 22.96it/s]








 82%|████████▏ | 4948/6000 [03:59<00:45,

 86%|████████▌ | 5173/6000 [04:10<00:40, 20.24it/s]








 86%|████████▋ | 5176/6000 [04:10<00:41, 19.86it/s]








 86%|████████▋ | 5178/6000 [04:10<00:41, 19.79it/s]








 86%|████████▋ | 5181/6000 [04:10<00:39, 20.49it/s]








 86%|████████▋ | 5184/6000 [04:10<00:42, 19.36it/s]








 86%|████████▋ | 5186/6000 [04:10<00:42, 18.99it/s]








 86%|████████▋ | 5188/6000 [04:11<00:42, 19.27it/s]








 87%|████████▋ | 5191/6000 [04:11<00:40, 19.91it/s]








 87%|████████▋ | 5194/6000 [04:11<00:39, 20.45it/s]








 87%|████████▋ | 5197/6000 [04:11<00:38, 20.61it/s]








 87%|████████▋ | 5200/6000 [04:11<00:37, 21.25it/s]








 87%|████████▋ | 5203/6000 [04:11<00:36, 21.74it/s]








 87%|████████▋ | 5206/6000 [04:11<00:37, 21.39it/s]








 87%|████████▋ | 5209/6000 [04:12<00:38, 20.37it/s]








 87%|████████▋ | 5212/6000 [04:12<00:37, 21.15it/s]








 87%|████████▋ | 5215/6000 [04:12<00:38, 20.39it/s]








 87%|████████▋ | 5218/6000 [04:12<00:38,

 90%|█████████ | 5429/6000 [04:22<00:29, 19.31it/s]








 91%|█████████ | 5432/6000 [04:23<00:28, 19.73it/s]








 91%|█████████ | 5435/6000 [04:23<00:27, 20.41it/s]








 91%|█████████ | 5438/6000 [04:23<00:27, 20.23it/s]








 91%|█████████ | 5441/6000 [04:23<00:27, 20.64it/s]








 91%|█████████ | 5444/6000 [04:23<00:25, 21.44it/s]








 91%|█████████ | 5447/6000 [04:23<00:26, 21.01it/s]








 91%|█████████ | 5450/6000 [04:23<00:27, 20.36it/s]








 91%|█████████ | 5453/6000 [04:24<00:29, 18.56it/s]








 91%|█████████ | 5456/6000 [04:24<00:28, 19.36it/s]








 91%|█████████ | 5458/6000 [04:24<00:28, 18.85it/s]








 91%|█████████ | 5460/6000 [04:24<00:29, 18.55it/s]








 91%|█████████ | 5462/6000 [04:24<00:29, 18.48it/s]








 91%|█████████ | 5465/6000 [04:24<00:27, 19.13it/s]








 91%|█████████ | 5467/6000 [04:24<00:27, 19.36it/s]








 91%|█████████ | 5469/6000 [04:24<00:28, 18.78it/s]








 91%|█████████ | 5472/6000 [04:25<00:26,

 95%|█████████▌| 5704/6000 [04:35<00:13, 21.84it/s]








 95%|█████████▌| 5707/6000 [04:36<00:13, 21.27it/s]








 95%|█████████▌| 5710/6000 [04:36<00:13, 20.78it/s]








 95%|█████████▌| 5713/6000 [04:36<00:15, 18.52it/s]








 95%|█████████▌| 5716/6000 [04:36<00:14, 19.21it/s]








 95%|█████████▌| 5718/6000 [04:36<00:15, 18.00it/s]








 95%|█████████▌| 5721/6000 [04:36<00:14, 19.30it/s]








 95%|█████████▌| 5723/6000 [04:36<00:14, 19.43it/s]








 95%|█████████▌| 5725/6000 [04:37<00:14, 18.57it/s]








 95%|█████████▌| 5728/6000 [04:37<00:13, 19.66it/s]








 96%|█████████▌| 5731/6000 [04:37<00:13, 20.39it/s]








 96%|█████████▌| 5734/6000 [04:37<00:13, 20.09it/s]








 96%|█████████▌| 5737/6000 [04:37<00:12, 20.59it/s]








 96%|█████████▌| 5740/6000 [04:37<00:12, 21.33it/s]








 96%|█████████▌| 5743/6000 [04:37<00:11, 22.00it/s]








 96%|█████████▌| 5746/6000 [04:38<00:11, 21.55it/s]








 96%|█████████▌| 5749/6000 [04:38<00:11,

100%|█████████▉| 5978/6000 [04:49<00:01, 18.29it/s]








100%|█████████▉| 5981/6000 [04:49<00:00, 20.23it/s]








100%|█████████▉| 5984/6000 [04:49<00:00, 20.72it/s]








100%|█████████▉| 5987/6000 [04:49<00:00, 20.92it/s]








100%|█████████▉| 5990/6000 [04:50<00:00, 19.35it/s]








100%|█████████▉| 5992/6000 [04:50<00:00, 18.54it/s]








100%|█████████▉| 5994/6000 [04:50<00:00, 18.84it/s]








100%|█████████▉| 5997/6000 [04:50<00:00, 19.50it/s]








100%|██████████| 6000/6000 [04:50<00:00, 20.65it/s]


In [138]:
data_with_states[1]

("the plan was that when the elephants charged , these lanes would open , allowing them to pass through the legionaries ' ranks and be dealt with at the rear of the army .",
 11,
 array([-1.06928237e-01, -8.40567425e-02,  1.79631397e-01,  2.97853976e-01,
        -2.15216964e-01, -2.51559556e-01,  5.60459018e-01,  4.81853276e-01,
         8.02331746e-01, -8.26776981e-01, -5.81259668e-01, -5.16310036e-01,
        -4.52610314e-01,  3.01652253e-02, -6.30002096e-02,  3.80439371e-01,
        -1.96840152e-01, -4.06810671e-01,  6.64438784e-01,  3.55722308e-01,
         2.11303487e-01,  5.95988095e-01, -1.98634103e-01,  3.71389806e-01,
         7.81558216e-01,  3.17605376e-01,  8.41748297e-01,  7.47532964e-01,
         1.06885374e-01,  2.75575399e-01,  3.52778971e-01,  2.21079409e-01,
        -3.38670224e-01, -5.22948384e-01,  6.87281251e-01,  4.90994275e-01,
        -4.45819683e-02,  1.63186759e-01,  2.61699945e-01, -3.63200307e-01,
        -1.03622004e-01, -1.59586459e-01,  6.86912298e-01,  2

In [139]:
random.shuffle(data_with_states)
l = int(len(data_with_states) * 0.8)
train, dev = data_with_states[:l], data_with_states[l:]
train_x, train_y = np.array([x[-2] for x in train]), np.array([x[-1] for x in train])
max_y = max([x[-1] for x in train])
dev_x, dev_y = np.array([x[-2] for x in dev]), np.array([x[-1] for x in dev])

In [140]:
model = LinearRegression()
model.fit(train_x, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [141]:
model.score(dev_x, dev_y)

0.4775728191273558

In [99]:
import pickle
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
import loss
import scipy
from scipy import linalg


class Dataset(torch.utils.data.Dataset):
    
    """Simple torch dataset class"""
    def __init__(self, data, device):

        self.data = data
        self.device = device

    def __len__(self):

        return len(self.data)

    def __getitem__(self, index):

        with torch.no_grad():
             
            vec1_np, vec2_np, str1, str2, _ = self.data[index]
            
            vec1, vec2, str1, str2, pair_id = self.data[index]            
            vec1, vec2 = torch.from_numpy(vec1_np).float(), torch.from_numpy(vec2_np).float()
            vec1 = vec1.to(self.device)
            vec2 = vec2.to(self.device)
            
            return (vec1, vec2, str1, str2, pair_id)
            
            
            
class Siamese(pl.LightningModule):

    def __init__(self, X_train, X_dev, dim, batch_size, dropout_rate, device = "cuda"):
        super(Siamese, self).__init__()
        self.l = torch.nn.Linear(1024, dim)
        
        self.train_data = Dataset(X_train, device)
        self.dev_data = Dataset(X_dev, device)
        self.train_gen = torch.utils.data.DataLoader(self.train_data, batch_size = batch_size, drop_last = False, shuffle=True)
        self.dev_gen = torch.utils.data.DataLoader(self.dev_data, batch_size = batch_size, drop_last = False, shuffle=True)
        self.loss_fn = loss.BatchHardTripletLoss(final = "softmax", device = device)
        self.dropout = torch.nn.Dropout(p = dropout_rate)
        self.acc = None
        self.optimizer = torch.optim.Adam(self.parameters(), weight_decay = 1e-6)
        
    def forward(self, x1, x2):

          h1 = self.l(x1)
          h2 = self.l(x2)
         
          return h1, h2
 
    def train_network(self, num_epochs):
    
      trainer = Trainer(max_nb_epochs = num_epochs, min_nb_epochs = num_epochs, show_progress_bar = False)
      trainer.fit(self)

      return self.acc   
      
    def get_weights(self):
    
        return self.l.weight.data.detach().cpu().numpy()
    
    def training_step(self, batch, batch_nb):
        # REQUIRED
        x1, x2, str1, str2, ids = batch
        h1, h2 = self.forward(self.dropout(x1), self.dropout(x2))
        loss_val =  self.loss_fn(h1, h2, str1, str2, ids, index=0, evaluation = False)
        
        return {'loss': loss_val[0]}
        

    def validation_step(self, batch, batch_nb):
    
        # OPTIONAL
        x1, x2, str1, str2, ids = batch
        h1, h2 = self.forward(x1, x2)
        loss_val =  self.loss_fn(h1, h2, str1, str2, ids, index=batch_nb, evaluation = True)
        return {'val_loss': loss_val[0]}

    def validation_end(self, outputs):
        # OPTIONAL    
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Loss is {}".format(avg_loss))
        return {'avg_val_loss': avg_loss}

    def configure_optimizers(self):
        # REQUIRED
        return torch.optim.Adam(self.parameters(), weight_decay = 1e-4)

    @pl.data_loader
    def train_dataloader(self):
        return self.train_gen

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        # can also return a list of val dataloaders
        return self.dev_gen

ModuleNotFoundError: No module named 'pytorch_lightning'